Information Visualization Project
===
Gabriela Malec
---

We will work with a dataset with information about collisions in New York City.
The Motor Vehicle Collisions crash table contains details on the crash event. Each row represents a crash event. The Motor Vehicle Collisions data tables contain information from all police reported motor vwe have detailed information about time and place of the collision, number of inehicle collisions in NYC. In our dataset jured people and contributing vehicles types.
We are specifically interested in the summers of 2018 and 2020.
Our goal is to create a static visualization that allows us to answer the following questions:
* Are accidents more frequent during weekdays or weekends? Is there any
difference between before COVID-19 and after?
* Is there any type of vehicle more prone to participate in accidents?
* At what time of the day are accidents more common?
* Are there any areas with a larger number of accidents?
* Is there a correlation between weather conditions and accidents?
* Are the accidents along the day are caused by the same type of vehicle?
* Does the number of pedestrians injured or dead depends on the type of vehicle?


Data Preprocessing
===
The first step to accurate and informative visualization is making sure our data is valid. As the first thing after downloading we uploaded our data to Open Refine to clean it.

We changed the date type to date and checked if we extracted the correct time period of our interest. Additionally, we changed all text values to Title Case and trimmed leading and trailing whitespace in all columns - we could observe major changes in the street names columns.

We made an attempt to cluster and merge values in the columns containing street names and vehicle types. We discovered that our data is very unorganized and there is many different ways to express the same street or vehicle type (i.e. 'amb', 'ambu', 'fdny amb' all meaning ambulance). We focused on unifying the most frequent values significant for our visualization.

One of the questions concerns the location of the accidents - we can observe many rows where the location is not available but we know the street of the accident. We use this information to retrieve the longitude and latitude for these crashes using Nominatim, a geocoding service provided by OpenStreetMap that we access through Geopy library in Python.

In [ ]:
import altair as alt
import pandas as pd
import geopandas as gpd
#from geopy.geocoders import Nominatim
#import math
#from geopy.extra.rate_limiter import RateLimiter

In [ ]:
crash = pd.read_csv('data-collisions-vehicles-new.csv')
alt.data_transformers.disable_max_rows()
crash.head()

In [ ]:
crash['STREET'] = crash['ON STREET NAME'].fillna(crash['OFF STREET NAME'])
crash['STREET'] = crash['STREET'].fillna(crash['CROSS STREET NAME'])

for index, row in crash.iterrows():
    if row['LATITUDE'] == 0.0:
        crash.at[index, 'LATITUDE'] = pd.NA

    if row['LONGITUDE'] == 0.0:
        crash.at[index, 'LONGITUDE'] = pd.NA

In [ ]:
'''
geolocator = Nominatim(user_agent="my_geocoder")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

for index, street_name in enumerate(crash["STREET"]):
    if pd.isnull(crash.at[index, 'LATITUDE']) and pd.isnull(crash.at[index, 'LONGITUDE']):
        location = geocode(f"{street_name}, New York City")
        if location:
            print(f"{street_name}: {location.latitude}, {location.longitude}")
            crash.at[index, 'LATITUDE'] = location.latitude
            crash.at[index, 'LONGITUDE'] = location.longitude
        else:
            print(f"Coordinates not found for {street_name}")

print(crash)
crash.to_csv('data-collisions-vehicles-new.csv', index=False)
'''

Additionally, there was 79 crash accidents without specified location. We decided to not include them in our visualization as their contribution to understanding of the data was insignificant.
Few locations were not retrieved properly and we corrected them by hand.
For some crashes our code was unable to retrieve the coordinates of the accident and they are left as NA's. Even though we will not be able to display those crashes on the map in the further visualization, we will use them to show the quantity of the total crashes.


Design process
====
We approached the visualization task by analysing it question by question. At first, we decided to create separate visualizations for each question to determine the best way to answer each one of them.

---
* Question 1
Are accidents more frequent during weekdays or weekends? Is there any
difference between before COVID-19 and after?

To answer this question, we need to consider additional data partition to before and after (we adress it as during) covid crashes.
The first idea we got was to use the bar chart where we represent the total number of crushes during the week vs during the weekend in separate bars.

In [ ]:
crash['CRASH DATE'] = pd.to_datetime(crash['CRASH DATE'])
crash['day_of_week'] = crash['CRASH DATE'].dt.day_name()
crash['weekday'] = crash['CRASH DATE'].dt.dayofweek < 5
crash['weekday'] = crash['weekday'].map({True: 'Weekday', False: 'Weekends'})

weekday_counts = crash.groupby('weekday').size().reset_index(name='count')

chart = alt.Chart(weekday_counts).mark_bar().encode(
    x=alt.X('weekday:N', title='Day of the week', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('count:Q', title='Number of accidents'),
    color=alt.Color('weekday:N', title='Day of the week', scale=alt.Scale(scheme='category10'), legend=alt.Legend(title=None)),
).properties(
    title='Accidents: weekdays and weekends',
    width=400
)

chart

Weekday has more days than the weekend so it is natural that more accidents occur during the week. Having that in mind, we decided to compare the average number of accidents during the week vs the weekend.

In [ ]:
crash['weekday'] = crash['CRASH DATE'].dt.dayofweek < 5
avg_accidents = crash.groupby(['CRASH DATE', 'weekday']).size().reset_index(name='count')
avg_accidents = avg_accidents.groupby('weekday')['count'].mean().reset_index(name='avg_count')
avg_accidents['weekday'] = avg_accidents['weekday'].map({True: 'weekday', False: 'weekend'})

chart = alt.Chart(avg_accidents).mark_bar().encode(
    x=alt.X('weekday:N', title='Day of the week', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('avg_count:Q', title='Average number of accidents per day'),
    color=alt.Color('weekday:N', title='Day of the week', scale=alt.Scale(scheme='category10'), legend=alt.Legend(title=None)),
    tooltip=['weekday:N', alt.Tooltip('avg_count:Q', format='.2f')]
).properties(
    title='Average number of accidents per day: weekdays vs weekends',
    width=400,
    height=600
)

chart

This bar already answers the first part of our question better, we can see that during the week the number of accidents is higher. The next step was to group our data to 'before covid'(2018) and 'during covid'(2020) times and repeat the plots for the grouped bar chart where the group cluster corresponds to the part of the week.

In [ ]:
covid_start_date = pd.to_datetime('2020-03-01', utc=True)
covid_end_date = pd.to_datetime('2023-05-05', utc=True)

crash['covid_period'] = 'before_covid'
crash.loc[(crash['CRASH DATE'] >= covid_start_date) & (crash['CRASH DATE'] <= covid_end_date), 'covid_period'] = 'during_covid'
crash.loc[crash['CRASH DATE'] > covid_end_date, 'covid_period'] = 'after_covid'

In [ ]:
crash['day_category'] = 'weekday'
crash.loc[crash['CRASH DATE'].dt.dayofweek >= 5, 'day_category'] = 'weekend'

count_accidents_combined = crash.groupby(['day_category', 'covid_period']).size().reset_index(name='count')

chart = alt.Chart(count_accidents_combined).mark_bar().encode(
    x=alt.X('covid_period:N', title=None, axis=alt.Axis(labelAngle=0)),
    y=alt.Y('count:Q', title='Number of Accidents'),
    color=alt.Color('covid_period:N', title='Time period', scale=alt.Scale(scheme='category10')),
    column=alt.Column('day_category:N', title=None, header=alt.Header(labels=True)),
    tooltip=['day_category:N', 'covid_period:N', 'count:Q']
).properties(
    title='Accidents: weekdays vs weekends - before and during covid',
    width=120
)

chart

In [ ]:
count_accidents_combined = crash.groupby(['covid_period', 'day_category', 'CRASH DATE']).size().reset_index(name='count')
avg_accidents_per_day = count_accidents_combined.groupby(['covid_period', 'day_category']).mean(numeric_only=False)['count'].reset_index(name='avg_accidents_per_day')

chart = alt.Chart(avg_accidents_per_day).mark_bar().encode(
    x=alt.X('covid_period:N', title='Period', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('avg_accidents_per_day:Q', title='Average accidents per day'),
    color=alt.Color('covid_period:N', title='Time period', scale=alt.Scale(scheme='category10')),
    column=alt.Column('day_category:N', title=None, header=alt.Header(labels=True)),
    tooltip=['day_category:N', 'covid_period:N', 'avg_accidents_per_day:Q']
).properties(
    title='Average accidents per day: weekdays vs weekends before and during COVID',
    width=120
)

chart

To see the ratio of average number of accidents happening during the week vs the weekend we plot a normalized stacked bars separately for two time periods - it helps us determine in very easy intuitive way when the accidents are more frequent and easily compare how the ratio changes between the time periods. The x-axis represents two periods in time, the y-axis shows the percentage of accidents per day, stacked to indicate the proportion between weekdays and weekends. The color scheme distinguishes between weekday and weekend categories. We need to be aware though that this bar takes a lot of space but we have decided that it is informative enough to include it in the final visualization as it gives us the best possibility of comparision between and within groups.

In [ ]:
count_accidents_combined = crash.groupby(['covid_period', 'day_category', 'CRASH DATE']).size().reset_index(name='count')
avg_accidents_per_day = count_accidents_combined.groupby(['covid_period', 'day_category']).mean(numeric_only=False)['count'].reset_index(name='avg_accidents_per_day')
avg_accidents_per_day['percentage'] = avg_accidents_per_day.groupby('covid_period')['avg_accidents_per_day'].transform(lambda x: x / x.sum() * 100)

chart = alt.Chart(avg_accidents_per_day).mark_bar().encode(
    x=alt.X('covid_period:N', title='Period', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('percentage:Q', title='Percentage of Accidents per Day', stack='normalize'),
    color=alt.Color('day_category:N', title='Day Category', scale=alt.Scale(scheme='blues'))
).properties(
    title='Average accidents per day: weekdays vs weekends - before and during COVID',
    width=300,
    height=600
)

chart

Additionaly to this question we wanted to have a closer look at each day of the week separately. We decide to plot a grouped bar chart where we present the total number of accidents for each of the days separately before and after covid. The x-axis represents two time periods, the y-axis indicates the count of collisions, and the color distinguishes between the periods, using a blue color scheme. The chart is faceted by the days of the week, providing a clear comparison of accident counts across different weekdays and time periods. The columns are sorted from Monday to Sunday for better readability.

In [ ]:
crash['CRASH DATE'] = pd.to_datetime(crash['CRASH DATE'], format='%Y-%m-%dT%H:%M:%SZ')
crash['Weekday'] = crash['CRASH DATE'].dt.day_name()

crash_count = crash.groupby(['covid_period', 'Weekday']).size().reset_index(name='crash_count')

q1 = alt.Chart(crash_count).mark_bar().encode(
    x=alt.X('covid_period:N', title='Period', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('crash_count:Q', title='Collisions Count'),
    color=alt.Color('covid_period:N', title='Covid period',scale=alt.Scale(scheme='blues')),
    column=alt.Column('Weekday:N', title='Day of Week', sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
).properties(
    width=alt.Step(40),
    height=400,
    title='Number of Accidents for Each Weekday'
).configure_view(stroke='transparent')
q1

As the similar alternative we considered plotting a grouped bar chart where we present the average number of accidents for each of the days separately before and after covid.
To not lose the information about general average number of accidents during the weekdays and weekends we could plot four vertical bars for weekdays and weekends before and after covid.

Another approach we wanted to try when visualizing number of accidents was a line plot with distinguished lines for weekday and weekend. First we plotted all data on one grid.

The x-axis represents the date, while the y-axis shows the count of crashes. The line color distinguishes between day categories (weekdays and weekends), and the line style indicates the COVID-19 period. The red dashed line marks the start of the COVID-19 period, providing a visual reference point.

 Because of the large blank space between the time periods in our data we decided to plot the time periods separately next to each other, making it easier to compare them. Representing data in this way gives us a good insight into the trends and how the number of collisions was changing over time before and after covid (within the groups). However, since it was not an original question we decided to not include it in our final design.

In [ ]:
chart = alt.Chart(count_accidents_combined).mark_line().encode(
    x=alt.X('CRASH DATE:T', title='Date'),
    y=alt.Y('count:Q', title='Number of Crashes'),
    color=alt.Color('day_category:N', title='Day Category', scale=alt.Scale(scheme='category10'), legend=alt.Legend(title='Day category')),
    strokeDash=alt.StrokeDash('covid_period:N', title='COVID Period', scale=alt.Scale(domain=['before_covid', 'during_covid'], range=[[0], []]), legend=None),
    tooltip=['CRASH DATE:T', 'count:Q', 'day_category:N', 'covid_period:N']
).properties(
    title='Number of crashes by day before and during COVID-19',
    width=800
)

rule = alt.Chart(pd.DataFrame({'start': [covid_start_date]})).mark_rule(color='red', strokeDash=[5]).encode(
    x='start:T',
    tooltip=['start:T']
)

label = alt.Chart(pd.DataFrame({'start': [covid_start_date], 'label': ['Start of COVID-19']})).mark_text(
    align='left',
    dx=5,
    dy=20,
    fontSize=12
).encode(
    x='start:T',
    y=alt.value(0),
    text='label'
)

combined_chart = chart + rule + label
combined_chart

In [ ]:
before_covid_data = count_accidents_combined[count_accidents_combined['covid_period'] == 'before_covid']
during_covid_data = count_accidents_combined[count_accidents_combined['covid_period'] == 'during_covid']

y_domain = [0, max(count_accidents_combined['count'])]

chart_before_covid = alt.Chart(before_covid_data).mark_line().encode(
    x=alt.X('CRASH DATE:T', title='Date'),
    y=alt.Y('count:Q', title='Number of Crashes', scale=alt.Scale(domain=y_domain)),
    color=alt.Color('day_category:N', title='Day category', scale=alt.Scale(scheme='blues')),
    tooltip=['CRASH DATE:T', 'count:Q', 'day_category:N']
).properties(
    title=' Number of crashes by day before COVID-19',
    width=400
)

chart_during_covid = alt.Chart(during_covid_data).mark_line().encode(
    x=alt.X('CRASH DATE:T', title='Date'),
    y=alt.Y('count:Q', title='Number of Crashes', scale=alt.Scale(domain=y_domain)),
    color=alt.Color('day_category:N', title='Day category', scale=alt.Scale(scheme='category10')),
    tooltip=['CRASH DATE:T', 'count:Q', 'day_category:N']
).properties(
    title='Number of crashes by day during COVID-19',
    width=400
)

combined_chart = alt.hconcat(chart_before_covid, chart_during_covid)
combined_chart

---
* Question 2 Is there any type of vehicle more prone to participate in accidents?

In order to answer the other question, we have to take a closer look at our columns containing information about vehicle types participating in the accidents. We see there is over 200 different vehicle types and since it would be very difficult to provide detailed information about all of them in our visualization, we decide to focus on the most significant vehicle types and treat the rest of types as the category 'others'.

We believe that the most adequate way to represent this data would be through bar chart where we encode in the height all the times where the specified vehicle type was the participant of the accident. The alrernative we considered was an enclosure diagram but for the consistency of our visualization we decided to choose the bar chart.

We hesitated if the minority of the vehicle types should be represented on the plot, but this count does not provide valuable information for our question as we want to focus on the most popular vehicle types. Additionaly, similarly to NA's, category 'others' in the middle of the plot does not look professional. We tried to move 'others' to tail by raising the number of distinguished vehicles. Unfortunetely it was unsuccesful attempt and it did not solve the problem of the category being in the middle of the chart, not at the end as the smallest value.
In the end, we decided to not include it in the barchart.

In [ ]:
#with other
vehicle_columns = ['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']

crash_long = pd.melt(crash, value_vars=vehicle_columns)

crash_long = crash_long.dropna(subset=['value'])

vehicle_counts = crash_long['value'].value_counts().reset_index()
vehicle_counts.columns = ['Vehicle Type', 'Count']

top_vehicle_types = vehicle_counts.head(10)['Vehicle Type'].tolist() #focusing on 10 most popular vehicle types
vehicle_counts['Vehicle Type'] = vehicle_counts['Vehicle Type'].where(vehicle_counts['Vehicle Type'].isin(top_vehicle_types), 'Other')

vehicle_counts = vehicle_counts.groupby('Vehicle Type')['Count'].sum().reset_index()

bar_chart = alt.Chart(vehicle_counts).mark_bar().encode(
    x=alt.X('Vehicle Type:N', title='Vehicle Type', axis=alt.Axis(labelAngle=-45), sort='-y'),
    y=alt.Y('Count:Q', title='Count')
).properties(
    title='Number of Accidents by Type of Vehicle',
    width=500
)

bar_chart

In [ ]:
#Without other 1

vehicle_columns = ['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']

crash_long = pd.melt(crash, value_vars=vehicle_columns)

crash_long = crash_long.dropna(subset=['value'])

vehicle_counts = crash_long['value'].value_counts().reset_index()
vehicle_counts.columns = ['Vehicle Type', 'Count']

top_vehicle_types = vehicle_counts.head(7)['Vehicle Type'].tolist()  # focusing on 10 most popular vehicle types
vehicle_counts['Vehicle Type'] = vehicle_counts['Vehicle Type'].where(vehicle_counts['Vehicle Type'].isin(top_vehicle_types), 'Other')

vehicle_counts_filtered = vehicle_counts[vehicle_counts['Vehicle Type'] != 'Other']

bar_chart = alt.Chart(vehicle_counts_filtered).mark_bar().encode(
    x=alt.X('Vehicle Type:N', title='Vehicle Type', axis=alt.Axis(labelAngle=-45), sort='-y'),
    y=alt.Y('Count:Q', title='Count')
).properties(
    title='Number of Accidents by Type of Vehicle',
    width=600
)

bar_chart

The plot we included in the final visualization represents the number of accidents for vehicle types with counts exceeding 2500. The x-axis displays different vehicle types, and the y-axis indicates the count of accidents. The bars are sorted in descending order, providing a clear comparison of accident frequencies across various vehicle types. This visualization helps identify the most common vehicle types involved in accidents, focusing on those with higher occurrence rates.

In [ ]:
#without other 2

crash['VEHICLE TYPE CODE 1'] = crash['VEHICLE TYPE CODE 1'].astype(str)
crash['VEHICLE TYPE CODE 2'] = crash['VEHICLE TYPE CODE 2'].astype(str)
crash['VEHICLE TYPE CODE 3'] = crash['VEHICLE TYPE CODE 3'].astype(str)
crash['VEHICLE TYPE CODE 4'] = crash['VEHICLE TYPE CODE 4'].astype(str)
crash['VEHICLE TYPE CODE 5'] = crash['VEHICLE TYPE CODE 5'].astype(str)

crash_long = pd.melt(crash, value_vars=['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'])

crash_long = crash_long.dropna(subset=['value'])

vehicle_counts = crash_long['value'].value_counts().reset_index()
vehicle_counts.columns = ['Vehicle Type', 'Count']

filtered_data = vehicle_counts[(vehicle_counts['Count'] >= 2500) & (vehicle_counts['Vehicle Type'] != 'nan')]

q2 = alt.Chart(filtered_data).mark_bar().encode(
    x=alt.X('Vehicle Type:N', title='Vehicle Type', axis=alt.Axis(labelAngle=-45), sort='-y'),
    y=alt.Y('Count:Q', title='Count')
).properties(
    title='Number of Accidents (over 2500) by Type of Vehicle'
)

q2

---
* Question 3
At what time of the day are accidents more common?

We decided that the line chart is the best way to answer this question. The data points are representing the frequency of accidents at different hours of the day. The X-axis displays the hours, and the Y-axis shows the count of accidents.

In [ ]:
hourly_vehicle_data = crash[['CRASH TIME']].copy()

hourly_vehicle_data['Hour'] = pd.to_datetime(hourly_vehicle_data['CRASH TIME'], format='%H:%M', errors='coerce').dt.hour

q3 = alt.Chart(hourly_vehicle_data
).mark_line(point=True).encode(
    x=alt.X('Hour', title='Crash Time'),
    y=alt.Y('count():Q', title='Count')
).properties(
    width=600,
    height=400,
    title='Accident frequency by time of day'
)

q3

Additionally, as we will see later, we have one more question (Question 6) about the number of crashes throughout the day, this time with distinguishing the model of the vehicle. To save space and to be able to compare vehicle-type specified number of accidents with all accidents, we decided to plot them together. We consider column 'Vehicle Type Code 1' as the type of a car that caused the accident - this time we do not use the cumulative count of all columns containing vehicle types to not outnumber the total count of collisions.
To include both separate values and the total count in the plot legend, we combine them into one dataframe.

The resulting line chart shows the trend of accidents throughout the day, with different lines representing various vehicle types and the total. The points on the lines indicate the actual counts at specific hours.
The legend on the right provides a color-coded key for different vehicle types, sorted based on their total counts.

In [ ]:
hourly_vehicle_data = crash[['CRASH TIME', 'VEHICLE TYPE CODE 1']].copy()

hourly_vehicle_data['Hour'] = pd.to_datetime(hourly_vehicle_data['CRASH TIME'], format='%H:%M', errors='coerce').dt.hour

top_5_vehicle_types = hourly_vehicle_data['VEHICLE TYPE CODE 1'].value_counts().nlargest(5).index  # 5 most popular car types

hourly_vehicle_data.loc[~hourly_vehicle_data['VEHICLE TYPE CODE 1'].isin(top_5_vehicle_types), 'VEHICLE TYPE CODE 1'] = 'Other'

grouped_data = hourly_vehicle_data.groupby(['Hour', 'VEHICLE TYPE CODE 1']).size().reset_index(name='Count')

total_crashes = hourly_vehicle_data.groupby('Hour').size().reset_index(name='Count')
total_crashes['VEHICLE TYPE CODE 1'] = 'Total'

combined_data = pd.concat([grouped_data, total_crashes], ignore_index=True)

combined_data_filtered = combined_data[combined_data['VEHICLE TYPE CODE 1'] != 'Other']

sort_order = combined_data_filtered.groupby('VEHICLE TYPE CODE 1')['Count'].sum().sort_values(ascending=False).index

chart = alt.Chart(combined_data_filtered).mark_line(point=True).encode(
    x=alt.X('Hour:O', title='Hour'),
    y=alt.Y('Count:Q', title='Number of Accidents'),
    color=alt.Color('VEHICLE TYPE CODE 1:N', title='Collision Categories' , scale=alt.Scale(scheme='tableau10', domain=list(sort_order)))
).properties(
    width=750,
    height=550,
    title='Line Plot of Accidents by Hour and Vehicle Type'
)

chart

---
* Question 4
 Are there any areas with a larger number of accidents?

 To determine the answer to this question, we wanted to use the available data and plot the map of NYC with spots of collisions. To achieve that, we prepared a background map using the GeoJSON data, projecting it with the Albers USA method. We created a layered visualization. This results in a map where the background represents geographical boundaries, and the points indicate specific locations on the map, representing collisions in New York City.
 Additionally we wanted to add color encoding for the number of people injured to determine if any areas are more dangerous.

From the map we see that collision locations are in general uniformly distributed within the regions and since in the most of the accidents nobody got injured there is no need for the additional color encoding.

Instead, we create the chorolpleth map with the regions based on the New York boroughs. We encode in the color of the borough the total number of collision that occured there in order to determine the neighbourhood with the most frequent accidents.


In [ ]:
borough_crash = crash.groupby('BOROUGH').size().reset_index(name='total_crashes')

borough_mapping = {
    'Staten Island': 5,
    'Queens': 4,
    'Brooklyn': 3,
    'Manhattan': 1,
    'Bronx': 2
}

borough_crash['boro_code'] = borough_crash['BOROUGH'].map(borough_mapping)

In [ ]:
gdf = gpd.read_file('test.geojson')
merged_gdf = gdf.merge(borough_crash, on='boro_code', how='left')
chart = alt.Chart(merged_gdf).mark_geoshape().encode(
    color=alt.Color('total_crashes:Q', title='Number of Collisions', scale=alt.Scale(scheme='yellowgreenblue')),
    tooltip=['boro_name:N', 'total_crashes:Q']
).project(
    type='albersUsa'
).properties(
    width=400,
    height=300,
    title="Number of collisions depending on the borough "
)
chart

For the better understanding of the data and to validate the information the choropleth map gives us we plotted bar plots of number of accidents for district of New York. We came to the conclusion that it does not give us much more information than the map, the only difference is the way of encoding the number of accidents - instead of using color in the bar plot we use height, which is more intuitive but map gives us a better idea of the actual locations of the accidents.

In [ ]:
crashes_boro = merged_gdf.sort_values(by='total_crashes', ascending=False)

chart = alt.Chart(crashes_boro).mark_bar().encode(
    alt.X('boro_name:N', title='Borough', sort='-y'),
    alt.Y('total_crashes:Q', title='Number of crashes'),
    color=alt.Color('total_crashes:Q', legend=None),
    tooltip=['boro_name:N', 'total_crashes:Q']
).properties(
    width=500,
    height=300,
    title="Collisions in boroughs"
)

chart

For the last experiment we tried to combine the number of accidents with the time of the accident to see if all districts follow the same trend in time.

We create a streamgraph that visually represents the total number of collisions over time, with each colored area representing a different borough.
The areas are stacked on top of each other, allowing us to see the overall trend of collisions while also understanding the contribution of each borough to the total count.
Hovering over the graph provides detailed information about the date, total crashes, and the specific borough at that point in time.
As the trends all the boroughs seem to be quite comparable, we decided to not include the plot in the final visualization because it does not provide any novel information and keep only the choropleth map.

In [ ]:
crash['crash_date'] = pd.to_datetime(crash['CRASH DATE'])

# data only before covid
crash_before_covid = crash[crash['crash_date'] < '2020-01-30']
total_accidents_borough_df = crash_before_covid.groupby(['crash_date', 'BOROUGH']).size().reset_index(name='total_crashes')

streamgraph = alt.Chart(total_accidents_borough_df).mark_area().encode(
    x='crash_date:T',
    y='total_crashes:Q',
    color='BOROUGH:N',
    order='BOROUGH:N',
    tooltip=['crash_date:T', 'total_crashes:Q', 'BOROUGH:N']
).properties(
    width=600,
    height=400,
    title="Total collisons over time"
)

streamgraph

---
* Question 5
Is there a correlation between weather conditions and accidents?

To answer the question about weather, we need to retrieve additional data about the wather in the New York City during the time period of our interest. We collect our data from the website https://www.weather.gov/okx/.

In [ ]:
crash['CRASH DATE'] = pd.to_datetime(crash['CRASH DATE'], utc = True)

In [ ]:
weather = pd.read_csv('weather.csv')
weather['DATE'] = pd.to_datetime(weather['DATE'], utc = True)
weather = weather[weather['NAME'] == 'NY CITY CENTRAL PARK, NY US']
columns_remove = ['DAPR', 'DASF', 'MDPR', 'MDSF', 'PGTM', 'PSUN', 'TAVG', 'TSUN', 'WDF2', 'WDF5', 'WESD', 'WESF', 'WSF2', 'WSF5',
                  'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT11', 'TOBS']
for column in columns_remove:
    del weather[column]

weather

In [ ]:
crashes_day = crash.groupby('CRASH DATE').size().reset_index(name='crashes_count')
merged_data = pd.merge(crashes_day, weather, left_on='CRASH DATE', right_on='DATE', how='inner')
merged_data['TAVG'] = (merged_data['TMAX'] + merged_data['TMIN']) / 2
merged_data

In [ ]:
crashes_day = crash.groupby('CRASH DATE').size().reset_index(name='crashes_count')
merged_data = pd.merge(crashes_day, weather, left_on='CRASH DATE', right_on='DATE', how='inner')
merged_data['TAVG'] = (merged_data['TMAX'] + merged_data['TMIN']) / 2
merged_data

After extracting the parts of the dataset that we considered significant for our question, we decided to create a heatmap to see if there is any correlation between weather conditions and collisions.
The resulting visualization is a correlation heatmap where the color intensity represents the strength and direction of the correlation between pairs of variables. Positive correlations are typically displayed in one color, while negative correlations are displayed in another color. The text annotations provide exact correlation values, and the color of the text indicates whether the correlation is above or below 0.5.


To our surprise, we cannot observe ant signicant connection. To safe space, we decided to keep it as the only visualization that gives us information about the weather in the final design.

In [ ]:
selected_columns = ['PRCP', 'TAVG', 'AWND', 'crashes_count']
selected_data = merged_data[selected_columns]

Correlation_matrix = selected_data.corr()
corr_df = Correlation_matrix.stack().reset_index(name='Correlation')
corr_df.columns = ['Variable-1', 'Variable-2', 'Correlation']
corr_df['Correlation'] = corr_df['Correlation'].round(2)

variable_names_mapping = {
    'PRCP': 'Precipitation',
    'TAVG': 'Temperature',
    'AWND': 'Wind',
    'crashes_count': 'Collisions'
}

corr_df['Variable-1'] = corr_df['Variable-1'].map(variable_names_mapping)
corr_df['Variable-2'] = corr_df['Variable-2'].map(variable_names_mapping)

heatmap = alt.Chart(corr_df).mark_rect().encode(
    x='Variable-1:N',
    y='Variable-2:N',
    color='Correlation:Q',
    tooltip=['Variable-1:N', 'Variable-2:N', 'Correlation:Q']
).properties(
    width=400,
    height=400,
    title='Correlation Heatmap'
)

text = alt.Chart(corr_df).mark_text(baseline='middle').encode(
    x='Variable-1:N',
    y='Variable-2:N',
    text='Correlation:Q',
    color=alt.condition(
        'datum.Correlation > 0.5',
        alt.value('white'),
        alt.value('black')
    )
)

text = text.encode(
    text=alt.Text('Correlation:Q', format='.2f'),
    tooltip=['Variable-1:N', 'Variable-2:N', 'Correlation:Q']
)

combined_chart = heatmap + text
combined_chart

To check the validity of the information from the correlation matrix, we still decided to check if we can see any trends in the scatterplots of weather conditions vs crashes count.
We create a set of scatterplots, each showing the relationship between the number of collisions and one of the selected weather variables. Each point on the scatter plot represents a specific date.
As suspected, we cannot identify any significant connections.

In [ ]:
variables = ['PRCP', 'TAVG', 'AWND']
charts = []

for variable in variables:
    chart = alt.Chart(merged_data).mark_circle().encode(
        x='crashes_count:Q',
        y=f'{variable}:Q',
        tooltip=['CRASH DATE:T', f'{variable}:Q', 'crashes_count:Q']
    ).properties(
        title=f'Correlation between {variable} and number of collisions'
    )

    charts.append(chart)

combined_chart = alt.hconcat(*charts)

combined_chart

In this plot we made an attempt to encode information about wind, rain and total crashes count.

The resulting scatter plot visually represents each data point as a circle on a graph.
The X-axis shows the number of crashes, the Y-axis shows the wind speed, and the size of each circle corresponds to the precipitation.
We can hover over data points to view additional information in the tooltip.

As in the case of other weather related plots, we did not find it informative enough to include it in the final visualization.

In [ ]:
chart = alt.Chart(merged_data).mark_circle().encode(
    x='crashes_count:Q',
    y='AWND:Q',
    size='PRCP:Q',
    tooltip=['crash_date:T', 'AWND:Q', 'PRCP:Q', 'crashes_count:Q']
).properties(
    title='Weather conditions and number of crashes'
)
chart

---
* Question 6
Are the accidents along the day are caused by the same type of vehicle?

Our first idea how to answer this question was to created a stacked bar chart for each hour of the day where in the total number of accidents at that time we distinguish the specific most popular types of cars that participate in the crashes.
We create a stacked bar chart that visualizes the distribution of accidents by hour and the involvement of the top most frequent vehicle types. Each bar represents the total number of accidents at a specific hour, segmented by different vehicle types. The color encodes the vehicle types, allowing for a quick comparison of their contributions to accidents throughout the day. The chart aims to identify patterns in the types of vehicles involved in accidents at different hours.

In [ ]:
hourly_vehicle_data = crash[['CRASH TIME', 'VEHICLE TYPE CODE 1']]

hourly_vehicle_data['Hour'] = pd.to_datetime(hourly_vehicle_data['CRASH TIME']).dt.hour

top_5_vehicle_types = hourly_vehicle_data['VEHICLE TYPE CODE 1'].value_counts().nlargest(5).index

hourly_vehicle_data.loc[~hourly_vehicle_data['VEHICLE TYPE CODE 1'].isin(top_5_vehicle_types), 'VEHICLE TYPE CODE 1'] = 'Other'

grouped_data = hourly_vehicle_data.groupby(['Hour', 'VEHICLE TYPE CODE 1']).size().reset_index(name='Count')

chart = alt.Chart(grouped_data).mark_bar().encode(
    x=alt.X('Hour:O', title='Hour'),
    y=alt.Y('Count:Q', title='Number of Accidents'),
    color=alt.Color('VEHICLE TYPE CODE 1:N', scale=alt.Scale(scheme='tableau10'))
).properties(
    width=600,
    title='Stacked Bar Chart of Accidents by Hour and Vehicle Type'
)

chart

As the next step we decided to create a line plot illustrating the variation in the number of accidents over the hours of the day, differentiated by the involvement of specific vehicle types. The 'Hour' axis represents the time of day, the 'Count' axis represents the number of accidents, and the colored lines depict different vehicle types. The chart is designed to highlight trends in accident occurrences throughout the day for the top 5 most frequent vehicle types, providing insights into their respective contributions to accidents during different hours. This chart inspired us to combine the overall accident count throughout the day with the incidents attributed to specific vehicle types.

In [ ]:
hourly_vehicle_data = crash[['CRASH TIME', 'VEHICLE TYPE CODE 1']]

hourly_vehicle_data['Hour'] = pd.to_datetime(hourly_vehicle_data['CRASH TIME']).dt.hour

top_5_vehicle_types = hourly_vehicle_data['VEHICLE TYPE CODE 1'].value_counts().nlargest(5).index

hourly_vehicle_data.loc[~hourly_vehicle_data['VEHICLE TYPE CODE 1'].isin(top_5_vehicle_types), 'VEHICLE TYPE CODE 1'] = 'Other'

grouped_data = hourly_vehicle_data.groupby(['Hour', 'VEHICLE TYPE CODE 1']).size().reset_index(name='Count')

chart = alt.Chart(grouped_data).mark_line().encode(
    x=alt.X('Hour:O', title='Hour'),
    y=alt.Y('Count:Q', title='Number of Accidents'),
    color=alt.Color('VEHICLE TYPE CODE 1:N', scale=alt.Scale(scheme='tableau10'))
).properties(
    width=750,
    height=550,
    title='Line Plot of Accidents by Hour and Vehicle Type'
)

chart

The next chart we created represents a stacked normalized bar chart illustrating the proportion of accidents throughout the day attributed to specific vehicle types. The top 5 most frequent vehicle types are highlighted, while others are grouped under 'Other.'

In [ ]:
hourly_vehicle_data = crash[['CRASH TIME', 'VEHICLE TYPE CODE 1']]

hourly_vehicle_data['Hour'] = pd.to_datetime(hourly_vehicle_data['CRASH TIME']).dt.hour

top_5_vehicle_types = hourly_vehicle_data['VEHICLE TYPE CODE 1'].value_counts().nlargest(5).index

hourly_vehicle_data.loc[~hourly_vehicle_data['VEHICLE TYPE CODE 1'].isin(top_5_vehicle_types), 'VEHICLE TYPE CODE 1'] = 'Other'

grouped_data = hourly_vehicle_data.groupby(['Hour', 'VEHICLE TYPE CODE 1']).size().reset_index(name='Count')

grouped_data['Proportion'] = grouped_data.groupby('Hour')['Count'].transform(lambda x: x / x.sum())

chart = alt.Chart(grouped_data).mark_bar().encode(
    x=alt.X('Hour:O', title='Hour'),
    y=alt.Y('Proportion:Q', title='Proportion of Accidents'),
    color=alt.Color('VEHICLE TYPE CODE 1:N',  scale=alt.Scale(scheme='tableau10'))
).properties(
    width=600,
    title='Stacked Normalized Bar Chart of Accidents by Hour and Vehicle Type'
)

chart

We decided to include in the final visualization the normalized bar chart as it gices us more additional information about the proportion between accidents caused by different types of vehicles throughout the day. The more quantitative approach abour the number of accidents is already included in the plot we created to answer question 3.

---
* Question 7
Does the number of pedestrians injured or dead depends on the type of vehicle?

The most intuitive way for us to approach this question is to create a bar chart. We decided on stacked bar chart which encodes the vehicle types on the x-axis, representing the 'Vehicle Type' with labels angled at -45 degrees to avoid overlapping of the text (as in case of using horizontal text). The y-axis represents the count of pedestrians affected by the accident, differentiated by color to distinguish between people who got injured and killed. We sorted the chart in descending order based on the count of pedestrians, providing a visual comparison of the severity of pedestrian outcomes across different vehicle types.


In [ ]:
pedestrian_data = crash[['VEHICLE TYPE CODE 1', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF PEDESTRIANS INJURED']]


top_10_vehicle_types = pedestrian_data['VEHICLE TYPE CODE 1'].value_counts().nlargest(10).index


top_10_data = pedestrian_data[pedestrian_data['VEHICLE TYPE CODE 1'].isin(top_10_vehicle_types)]


grouped_data = top_10_data.groupby('VEHICLE TYPE CODE 1').sum().reset_index()

melted_data = pd.melt(grouped_data, id_vars=['VEHICLE TYPE CODE 1'], value_vars=['NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF PEDESTRIANS INJURED'],
                      var_name='Pedestrian Status', value_name='Count')

chart = alt.Chart(melted_data).mark_bar().encode(
    x=alt.X('VEHICLE TYPE CODE 1:N', title='Vehicle Type', axis=alt.Axis(labelAngle=-45), sort=alt.SortField(field='Count', order='descending')),
    y=alt.Y('Count:Q', title='Count'),
    color=alt.Color('Pedestrian Status:N',
                    legend=alt.Legend(title=None, orient='bottom', labelLimit=0, symbolLimit=0, symbolType='square'),
                    scale=alt.Scale(scheme='blues')
    )
).properties(
    width=600,
    title='Number of Pedestrians Injured and Killed by Vehicle Type (Top 10 Types)'
)

chart

We display the chosen visualization using multiple views in streamlit. Working with streamlit turned out to be the biggest challange for us, due to many technical problems. The final visualization might not be perfect but we put a lot of effort to keep it as consistent as possible. Some charts are slightly overlapping and we were not able to prevent it. However, all of them can be enlarged to access them fully.

Regarding the map, we wanted to show the amount of accidents in every district of New York by coloring the districts. Therefore, we had to merge the crash dataset with the map data of the geojson file. Since we had to access the geojson file via a link on Github, the merge operation in Streamlit was not possible anymore.


Answers to questions through final visualization
==
Based on our final visualization we aim to answer the original questions of the task.


* Question 1

Are accidents more frequent during weekdays or weekends?

Among the charts we can see a grouped bar chart with the number of collisions divided into periods before and during covid and differentiated by days of the week. There is also a stacked bar chart created by using values of average number of collisions per day. As we can see, accidents are more frequent during weekdays for both periods. However, what has changed is that during covid, the difference between accidents on weekdays and weekends began to decrease.

* Question 2

Is there any type of vehicle more prone to participate in accidents?


To answer this question we can look at the bar chart showing the number of accidents for vehicle types with counts exceeding 2500. It is clearly visible that the highest bar was achieved by "Sedan" and therefore we can say that this type of vehicle is more prone to participate in accidents.


* Question 3

At what time of the day are accidents more common?

In the line chart we can see the accidents by hour (on the x axis) and type of vehicle (on the y axis). When we look at the blue line, which shows the total amount of accidents by hour, we can say that accidents are more common between 8am and 8pm with a peak at 4pm.


* Question 4

Are there any areas with a larger number of accidents?

In the map we can see the location of accidents in New York City visualized by blue plots. We can see that the most accidents are happening in Brooklyn, Queens, Manhattan and Bronx.
Additionally, we can see that most of the accidents did not cause a lot of harm to people and the most serious accidents are scattered around the whole New York without any specific pattern.

The visualization is in our colab notebook that we could not include in our streamlit app gives a better overview on the accidents per district.
From the mentioned map we can not only see the order of the boroughs based on the number of collisions (from the color encoding) but by sliding over each district we can see the exact number of accidents. The most collisions, that is 25100 of them took place in Brooklyn.


* Question 5

Is there a correlation between weather conditions and accidents?

By looking at the correlation matrix, we see that the most typical weather conditions do not seem to have significant impact on the accidents count. The biggest correlation of our interest would be with wind - as it is negative correlation, we deduct that the more windy it is the least collisions occur.
We can see that precipitation has a slight positive impact on the number of collisions, during the rainy days collisions are more likely to happen.
Temperature does not have much impact on the number of crashes.

* Question 6

Are the accidents along the day are caused by the same type of vehicle?

By looking at the chart 'Line Plot of Accidents by Hour and Vehicle Type' we can distinguish for the most popular Vehicle Types the similar trend. For the whole day the biggest contributor to the total collision count is Sedan, followed by Station Wagon or Sport Utility Vehicle. For the less frequent car types the plot is more difficult to read as the lines overlap.

By looking at the 'Stacked Bar Chart of Accidents by Hour and Vehicle Type' we see that throught the day the proportion of the collisions caused by less popular vehicle types slightly changes. During the night and early morning hours (around 21-5) one of the most significant contributors is Taxi, during the day hours the proportion changes and collisions are more often caused by Box Trucks and Pick-up Trucks.

We see that there is still a significant proportion of other vehicles causing collisions throughout the whole day.


* Question 7

Does the number of pedestrians injured or dead depends on the type of vehicle?

From the plot "Number of Pedestrians Injured and Killed by Vehicle Type (Top 10 Types)' we can read that indeed the number of pedestrians injured or dead depends on the type of vehicle. The majority of harm was caused by the vehicle type Sedan, followed by Station Wagon or Sport Utility Vehicle. The other types of vehicles have caused significantly less damage. By looking at the number of accidents caused by the specific vehicle type, we see that these two counts strictly correlate - the more accidents the car type causes, the more harm to pedestrians happens.


